# Работа с датой и временем
Даты и время (тип `datetime`) часто встречаются во время предобработки данных для машинного обучения, будь то время конкретной продажи или год сбора каких-либо статистических данных общественного здравоохранения. Рассмотрим стратегии обработки временных рядов, включая решение проблемы, связанной с часовыми поясами и создание признаков с запаздыванием во времени с помощью инструментов временных рядов библиотеки pandas.

***

# Конвертировать строки в метки datetime

### 1. Конвертирование строковых значений в даты
##### Задача
Дан вектор строк, представляющих даты и время, требуется преобразовать их в данные временных рядов
##### Решение
Используем функцию `to_datetime()` библиотеки pandas с параметром `format=`

In [40]:
# Загрузить библиотеки
import pandas as pd 
import numpy as np

# Создать строки
date_strings = np.array([
                    '02-03-2019 11:35 PM',
                    '23-05-2018 12:01 AM',
                    '30-05-2011 07:55 PM',
                ])

# Конвертировать строки в метки datetime
[pd.to_datetime(date, format='%d-%m-%Y %I:%M %p') for date in date_strings]

[Timestamp('2019-03-02 23:35:00'),
 Timestamp('2018-05-23 00:01:00'),
 Timestamp('2011-05-30 19:55:00')]

Возможно возникнет необходимость добавить аргумент в параметр `errors=` для устранения проблем:

In [4]:
[pd.to_datetime(date, format='%d-%m-%Y %I:%M %p', errors='coerce') for date in date_strings]

[Timestamp('2019-03-02 23:35:00'),
 Timestamp('2018-05-23 00:01:00'),
 Timestamp('2011-05-30 19:55:00')]

Если задан параметр `errors='coerce'`, то любая возникшая проблема не вызовет ошибку (поведение по умолчанию), а вместо этого установит значение, вызывающее ошибку равным `NaT` ( от англ. – $Not$ $a$ $Time$).

Когда даты и время поступают в виде строковых значений, требуется их преобразовать в тип данных, который Python сможет понять. Хотя существует ряд инструментов самого языка Python для преобразования строк в тип `datetime`, для ускорения процесса приведения типов рекомендуется применять функцию `to_datetime()` библиотеки pandas.

Одним из препятствий для преобразования строк в дату и время является то, что формат записи значений даты и времени в строку может отличаться от источника к источнику данных. Например, один вектор дат может быть записан в формате: `23 марта 1997 г.`, другой как `03-23-97`, а третий – `23/3/1997` и т.д. Для указания точного соответствия текстовому формату записи используется парметр `format=`.

| Код | Описание | Пример |
|-----|----------|--------|
|%Y|Полный год|2001|
|%m|Месяц с дополнительным нулем|04|
|%d|День месяца с дополнительным нулем|09|
|%I|Час (12-ти часовое измерени) с дополнительным нулем|02|
|%p|AM (до полудня) или PM (после полудня)|AM|
|%M|Минута с дополнительным нулем|06|
|%S|Секунда с дополнительным нулем|05|

##### Дополнительные материалы
* [Полный список строковых кодов Python](https://strftime.org/)

### 2. Обработка часовых поясов
##### Задача
Дан временной ряд, требуется добавить или изменить информацию о часовом поясе
##### Решение
Если не указано иное, объекты pandas часового пояса не имеют. Вместе с тем можно добавить часовой пояс с помощью парметра `tz=` во время создания:

In [5]:
# Создать метку datetime
pd.Timestamp('2020-01-18 11:47', tz='Europe/Moscow')

Timestamp('2020-01-18 11:47:00+0300', tz='Europe/Moscow')

Часовой пояс можно добавить к ранее созданной метке времени `datetime` с помощью метода `tz_localize()`:

In [6]:
# Создать метку datetime
date = pd.Timestamp('2020-01-01 01:01')

# Задачть часовой пояс
date_in_Moscow = date.tz_localize('Europe/Moscow')

# Показать метку datetime
date_in_Moscow

Timestamp('2020-01-01 01:01:00+0300', tz='Europe/Moscow')

Также можно выполнить преобразование в другой часовой пояс:

In [7]:
# Изменить часовой пояс
date_in_Moscow.tz_convert('Europe/London')

Timestamp('2019-12-31 22:01:00+0000', tz='Europe/London')

Объекты класса `Series` библиотеки pandas могут применять методы `tz_localize()` и `tz_convert()` к каждому элементу:

In [11]:
# Создать три даты
dates = pd.Series(pd.date_range('2/2/2019', periods=3, freq='M'))

# Задать часовой пояс
dates.dt.tz_localize('Africa/Abidjan')

0   2019-02-28 00:00:00+00:00
1   2019-03-31 00:00:00+00:00
2   2019-04-30 00:00:00+00:00
dtype: datetime64[ns, Africa/Abidjan]

Библиотека pandas поддерживает два набора строковых значений, представляющих часовые пояса, например строки библитеки $pytz$. импортировав массив `all_timezones`, можно увидеть все строки, используемые для представления часовых поясов:

In [13]:
# Загрузить библиотеку
from pytz import all_timezones

# Показать часовые пояса
all_timezones[:5]

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara']

### 3. Выбор дат и времени
##### Задача
Дан вектор дат, требуется выбрать одну дату или несколько
##### Решение
Используем условные выражения в качестве начальной и конечной даты

In [14]:
# Создать фрейм данных
df = pd.DataFrame()

# Создать метки datetime
df['дата'] = pd.date_range('1/1/2001', periods=10000, freq='H')

# Выбрать наблюдения между двумя метками datetime
df[(df['дата'] > '2002-1-1 01:00:00') & (df['дата'] < '2002-1-1 05:34:21')]

,дата
8762,2002-01-01 02:00:00
8763,2002-01-01 03:00:00
8764,2002-01-01 04:00:00
8765,2002-01-01 05:00:00


В качестве альтернативы можно установить столбец даты как индекс фрейма данных, а затем сделать срез с помощью метода `loc()`:


In [17]:
# Задать индекс
df = df.set_index(df['дата'])

# Выбрать наблюдения между датами
df.loc['2001-02-03 11:00:00':'2001-02-03 14:34:00']

,дата
дата,
2001-02-03 11:00:00,2001-02-03 11:00:00
2001-02-03 12:00:00,2001-02-03 12:00:00
2001-02-03 13:00:00,2001-02-03 13:00:00
2001-02-03 14:00:00,2001-02-03 14:00:00


Какую стратегию использовать условные выражения или индексация по дате – определят ситуация. Если требуется выполнить сложные манипуляции с временными рядами, установка столбца даты в качестве индекса фрейма данных может вылиться в накладные расходы, но если нужно сделать простую проверку даты, то условыне выражения мошут оказаться проще.

### 4. Разбиение даты на несколько признаков
##### Задача
Дан столбец дат и времени, требуется создать признаки для года, месяца, дня, часа и минуты
##### Решение
    Используем свойства времени объекта-получателя `Series.dt` библиотеки pandas

In [18]:
# Создать фрейм данных
df = pd.DataFrame()

# Создать пять дат
df['date'] = pd.date_range('1/1/2019', periods=150, freq='W')

# Создать признаки для года, месяца, дня, часа и минуты
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute

# Показать строки
df.head()

,date,year,month,day,hour,minute
0,2019-01-06,2019,1,6,0,0
1,2019-01-13,2019,1,13,0,0
2,2019-01-20,2019,1,20,0,0
3,2019-01-27,2019,1,27,0,0
4,2019-02-03,2019,2,3,0,0


Иногда бывает полезно разбить столбец дат на компоненты. Например, может понадобиться признак, который включает только год наблюдения, либо возникает необходимость посмотреть на данные в помесячном разрезе.

### 5. Вычисление разницы между датами
##### Задача
Даны два признака `datetime`, требуется для каждого наблюдения рассчитать время между ними
##### Решение
Вычтем один признак `datetime` из другого

In [25]:
# Создать фрейм данных
df = pd.DataFrame()

# Создать два признака datetime
df['income'] = [pd.Timestamp('17-10-2019'), pd.Timestamp('09-11-2019')]
df['outcome'] = [pd.Timestamp('26-12-2019'), pd.Timestamp('17-01-2020')]

# Вычислить продолжительность между признаками
df['outcome'] - df['income']

0    70 days
1   128 days
dtype: timedelta64[ns]

Часто требуется удалить из результат `days` и оставить только числовое значение

In [26]:
# Вычислить продолжительность между признаками
pd.Series(delta.days for delta in (df['outcome'] - df['income']))

0     70
1    128
dtype: int64

Также бывает, что требуемы признак является разностью между двумя точками во времени. Например, если имеются даты заезда и выезда клиента из гостиницы, то искомый признак – это продолжительность пребывания клиента. Для таких расчетов рекомендуется использовать тип данных `TimeDelta`
##### Дополнительные материалы
* [Документация pandas: дельта времени](https://pandas.pydata.org/pandas-docs/stable/user_guide/timedeltas.html)

### 6. Кодирование дней недели
##### Задача
Дан вектор дат, требуется узнать день недели каждой даты
##### Решение
Используем атрибут `weekday_name` объекта получателя `Series.dt` библиотеки pandas

In [30]:
# Создать даты
dates = pd.Series(pd.date_range('1/1/2015', periods=5, freq='Y'))

# Показать дни недели
dates.dt.weekday_name

0    Thursday
1    Saturday
2      Sunday
3      Monday
4     Tuesday
dtype: object


В том случае, когда необходим результат с числовым значением, т.е. более пригодным в качестве признака для задач машинного обучения, необходимо использовать атрибут `weekday`, где дни недели представлены в виде целого числа (понедельник равен $0$):

In [32]:
# Показать дни недели
dates.dt.weekday

0    3
1    5
2    6
3    0
4    1
dtype: int64

Знание дня недели может быть полезным, елси, например, требвется сравнить общий объем продаж по воскресениям за последнии три года. Библиотека pandas упрощает создание вектора признаков, сожержащих информацию о будних днях.
##### Дополнительные материалы
* [Перечень datetime-подобных свойств объекта Series библиотеки pandas](https://pandas.pydata.org/pandas-docs/stable/reference/index.html)

### 7. Создание запаздывающего признака
##### Задача
Требуется создать признак, который запаздывает на $n$ периодов времени
##### Решение
Используем метод `shift()` библиотеки pandas

In [33]:
# Создать дата фрейм
df = pd.DataFrame()

# Создать дату
df['dates'] = pd.date_range('3/3/2005', periods=5, freq='D')
df['stock prices'] = [1.1, 2.2, 3.3, 4.4, 5.5]

# Задачть значения с запазданием на одну строку
df['stock prices on the previous day'] = df['stock prices'].shift(1)

# Показать фрейм данных
df

,dates,stock prices,stock prices on the previous day
0,2005-03-03,1.1,NaN
1,2005-03-04,2.2,1.1
2,2005-03-05,3.3,2.2
3,2005-03-06,4.4,3.3
4,2005-03-07,5.5,4.4


Часто данные основаны на регулярных интервалах времени. Например, данные за каждый день, час, минуту и т.д. Чтобы делать предсказания необходимо использовать значения в прошлом (так называемое "задерживание признака во времени"). Например, может возникнуть необходимость предсказать новую цену акций на основе цены, которая была накануне. Для этого можно применить метод `shift()` библиотеки pandas для задержания значений во времени на одну строку, создавая новый признак, содержащий прошлые значения.

В решении выше первая строка признака `stock prices on the previous day` является пропущенным значением, поскольку предыдущее значение цены акций `stock prices` отсутствует.

### 8. Использование скользящих временных окон
##### Задача
Дан временной ряд, требуется рассчитать некоторый статистический показатель для скользящего времени
##### Решение
Используем метод `rolling()` библиотеки pandas

In [34]:
# Создать метки datetime
time_index = pd.date_range('1/1/2010', periods=5, freq='M')

# Создать фрейм данных, задать индекс
df = pd.DataFrame(index=time_index)

# Создать признак
df['stock prices'] = [1, 2, 3, 4, 5]

# Вычислить скользящее среднее
df.rolling(window=2).mean()

,stock prices
2010-01-31,NaN
2010-02-28,1.5
2010-03-31,2.5
2010-04-30,3.5
2010-05-31,4.5


Скользящие (так называемые перемещающиеся) временные окна концептуально просты. Например, имеются ежемесячные наблюдения за ценой акций. Часто бывает полезно иметь временнóе окно по наблюдениям продолжительностью в заданное количество месяцев, а затем перемещаться по наблюдениям, вычисляя статистический показатель для всех наблюдений во временном окне.

Например, если есть временнóе окно продолжительностью три месяца, и требуется вычислить скользящее среднее, то вычисляется это следующим образом:
* средее(январь, февраль, март)
* среднее(февраль, март, апрель)
* среднее(март, апрель, май)
и т.д.

Другими словами трехмесяцное окно "скользит" по наблюдениям, вычисляя среднее значение на каждом шаге.

Метод `rolling()` библиотеки pandas мозволяет задавать размер окна с помощью парметра `window=`, а затем быстро рассчитывать некоторые распространенные статистические показатели, например:
* максимальное значение (`max()`);
* среднее значение (`mean()`);
* количество значений (`count()`);
* скользящую корреляцию (`corr()`).

Скользящие средние часто используются для сглаживания данных временных рядов, поскольку использование среднего значения всего временного ряда ослабляет эффект краткосрочныз колебаний.
##### Дополнительные материалы
* [Документация pandas: скользящие окна](https://pandas.pydata.org/pandas-docs/stable/user_guide/computation.html)
* ["Что такое скользящее окно и методы сглаживания" Справочник по инженерной статистике](https://www.itl.nist.gov/div898/handbook/pmc/section4/pmc42.htm)

### 9. Обработка пропущеных дат во временном ряду
##### Задача
В данных временых рядов пропущены значения. ОБработать эту ситуацию
#### Решение
Используем метод `interpolate()` библиотеки pandas для заполнения промежутков пропущеных значений

In [36]:
# Создать дату
time_index = pd.date_range('1/1/2010', periods=5, freq='M')

# Создать фрейм данных, задать индекс
df = pd.DataFrame(index=time_index)

# Создать признак с промежутком пропущеных значений
df['costs'] = [1.0, 2.0, np.nan, np.nan, 5]

# Интерполировать пропущенные значения
df.interpolate()

,costs
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,3.0
2010-04-30,4.0
2010-05-31,5.0


Другой вариант заполнить пропущенные значения последним перед пустым промежутком известным значением, т.е. выполнить прямое заполнение:

In [37]:
# Прямое заполнение
df.ffill()

,costs
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,2.0
2010-04-30,2.0
2010-05-31,5.0


Еще одним вариантом является заполнение пропущеных значений последним после пустого промежутка известным значением, т.е. выполнить обратное заполнение:

In [38]:
# Обратное заполнение
df.bfill()

,costs
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,5.0
2010-04-30,5.0
2010-05-31,5.0


***Интерполяция*** – этот метод заполнения промежутков, вызванных пропущенными значениями, путем по сути, построенияпрямой линии или кривой между известными значениями граничащими с промежутком, и использования этой прямой или кривой для предсказания разумных значений. Интерполяция может быть в особенности полезна, когда промежутки времени между точками постоянны, данные не подвержены шумовым колебаниям, а промежутки, вызванные пропусками, невелики. В приведенном выше примере промежуток из двух пропущенных значений граничит с `2.0` и `5.0`. Путем подгонки прямой, начинающейся с $2.0$ и заканчивающейся в $5.0$, можно сделать разумные предположения для двух пропущенных значений между `3.0` и `4.0`.

В случае, когда прямая между двумя точками считается нелинейной, необходимо указать тип предполагаемой криой с помощью парметра `method=` для метода `interpolate()`:

In [41]:
# Интерполировать пропущенные значения
df.interpolate(method='quadratic')

,costs
2010-01-31,1.000000
2010-02-28,2.000000
2010-03-31,3.059808
2010-04-30,4.038069
2010-05-31,5.000000


Наконец, могут быть случаи, когда имеются большие промежутки пропущенных значений, и необходимо интерполировать значения только по части промежутка. Для этого необходимо использовать параметр `limit=` для ограничения количества интеполируемых значений, и парметр `limit_direction=` для задания, следует ли интерполировать значения вперед от последнего передпромежутком известного значения, или наоборот:

In [43]:
# Интерполировать пропущенные значения
df.interpolate(limit=1, limit_direction='forward')

,costs
2010-01-31,1.0
2010-02-28,2.0
2010-03-31,3.0
2010-04-30,NaN
2010-05-31,5.0


Обратное заполнение и прямое заполнение можно рассматривать, как формы интерполяции, где интерполируемые значения находятся на константной прямой парраельной оси $Ox$, константой которой является ближайшее известное значение или перед или после пустого промежутка.